In [13]:
module StaticModel

using Base: Float64
using Scruff
using Scruff.Utils
using Scruff.RTUtils
using Scruff.Models
using Scruff.SFuncs
using Scruff.Algorithms

needsMaintenanceSF = Cat([true, false], [0.5, 0.5])

tempSF = Chain(Tuple{Bool}, Float64, tuple -> begin 
    nmBool = tuple[1]
    if nmBool 
        Uniform(0.0, 300.0)
    else
        Normal(207.5, 4.0)
    end
end)

nmModel = SimpleModel(needsMaintenanceSF)
needsMaintenance = nmModel(:needsMaintenance)

tempModel = SimpleModel(tempSF)
temperature = tempModel(:temp)

variables = [temperature, needsMaintenance]
graph = VariableGraph(temperature => [needsMaintenance])
net = InstantNetwork(variables, graph)
runtime = Runtime(net)
numSamples = 10000
alg = LW(numSamples)

observedTemp = 207.5
score = HardScore(observedTemp)
evidence = Dict{Symbol, Score}(:temp => score)
sampledResults = infer(alg, runtime, evidence)

probNeedsMaintenance = probability(sampledResults, x -> begin 
    x[:needsMaintenance]
end)

probTemp = probability(sampledResults, x -> begin 
    199 < x[:temp] < 201
end)


# println(probTemp)
# println(probNeedsMaintenance)

end

Main.StaticModel

In [14]:
module DynamicModel2

using Base: Float64
using Scruff
using Scruff.Utils
using Scruff.RTUtils
using Scruff.Models
using Scruff.SFuncs
using Scruff.Algorithms
import Scruff: make_initial, make_transition

struct MaintenanceModel <: VariableTimeModel{Tuple{}, Tuple{Bool}, Bool}
end

make_initial(::MaintenanceModel, t) = Cat([true, false], [0.5, 0.5])
make_transition(::MaintenanceModel, parts, t) = 
    Chain(Tuple{Bool}, Bool, tuple -> begin 
        needsMaintenance = tuple[1]
        Mixture(
            [Constant(needsMaintenance), Cat([true, false], [0.5, 0.5])],
            [0.9, 0.1]
        )
    end)

struct TemperatureModel <: VariableTimeModel{Tuple{}, Tuple{Bool}, Float64}
end
make_initial(::TemperatureModel, t) = Normal(207.5, 4.0)
make_transition(::TemperatureModel, parts, t) = 
    Chain(Tuple{Bool}, Float64, tuple -> begin 
        nmBool = tuple[1]
        # no dependence on previous temperature results in unstable temperature inference
        if nmBool 
            Uniform(0.0, 300.0)
        else
            Normal(207.5, 4.0)
        end
    end)

function run_inference()
    needsMaintenance = MaintenanceModel()(:needsMaintenance)
    temperature = TemperatureModel()(:temp)
    variables = [needsMaintenance, temperature]
    graph = VariableGraph(
        temperature => [needsMaintenance], 
        needsMaintenance => [needsMaintenance]
    )
    net = DynamicNetwork(variables, VariableGraph(), graph)
    runtime = Runtime(net)
    numParticles = 10000
    pf = AsyncPF(numParticles, numParticles, Int)
    init_filter(pf, runtime)

    score = HardScore{Float64}(207.5)
    for t in 1:30
        evidence = 
            if t == 1 
                Dict{Symbol, Score}(:temp => score)
            else
                Dict{Symbol, Score}()
            end
        filter_step(pf, runtime, [needsMaintenance, temperature], t, evidence)
        sampledResults = get_state(runtime, :particles)
        probNeedsMaintenance = probability(sampledResults, x -> begin 
            x[Symbol("needsMaintenance_$t")]
        end)
        # println("results at time $t")
        # println(probTemp)
        println(probNeedsMaintenance)

    end
end 

# run_inference()

end

Main.DynamicModel2

In [11]:
module DynamicModel3

using Base: Float64
using Scruff
using Scruff.Utils
using Scruff.RTUtils
using Scruff.Models
using Scruff.SFuncs
using Scruff.Algorithms
import Scruff: make_initial, make_transition

# add simple change in temperature 
# show prognostics 
# tweak slide languages for example 3 to reflect this 

temp_mean = 207.5
temp_var = 4.0

struct MaintenanceModel <: VariableTimeModel{Tuple{}, Tuple{Bool}, Bool}
end

make_initial(::MaintenanceModel, t) = Cat([true, false], [0.5, 0.5])
make_transition(::MaintenanceModel, parts, t) = 
    Chain(Tuple{Float64}, Bool, tuple -> begin 
        temp = tuple[1]
        if temp - temp_mean > temp_var
            Constant(true)
        else
            Constant(false)
        end
    end)

struct TemperatureModel <: VariableTimeModel{Tuple{}, Tuple{Bool, Float64}, Float64}
end
make_initial(::TemperatureModel, t) = Normal(temp_mean, temp_var)
make_transition(::TemperatureModel, parts, t) = 
    Chain(Tuple{Float64}, Float64, tuple -> begin 
        prevTemp = tuple[1]
        Normal(prevTemp + 1, temp_var)
    end)

function run_inference()
    needsMaintenance = MaintenanceModel()(:needsMaintenance)
    temperature = TemperatureModel()(:temp)
    variables = [needsMaintenance, temperature]
    graph = VariableGraph(
        temperature => [temperature], 
        needsMaintenance => [temperature]
    )
    net = DynamicNetwork(variables, VariableGraph(), graph)
    runtime = Runtime(net)
    numParticles = 10000
    pf = AsyncPF(numParticles, numParticles, Int)
    init_filter(pf, runtime)

    observedTemp = 207.5
    for t in 1:100
        score = HardScore(observedTemp)
        evidence = 
            if t == 1
                Dict{Symbol, Score}(:temp => score)
            else
                Dict{Symbol, Score}()
            end
        filter_step(pf, runtime, [needsMaintenance, temperature], t, evidence)
        sampledResults = get_state(runtime, :particles)
        
        probTemp= probability(sampledResults, x -> begin 
            abs(x[Symbol("temp_$t")] - observedTemp) < 2.0
        end)
        # println("results at time $t")
#         println(probTemp)
        probNeedsMaintenance = probability(sampledResults, x -> begin 
            x[Symbol("needsMaintenance_$t")]
        end)
        println(probNeedsMaintenance)

    end
end 

# run_inference()

end

Main.DynamicModel3